In [ ]:
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os


In [ ]:
f1_scores_list = []
f1_scores_list_random = []
seismic_acc_list = []
seismic_to_all_anom_list = []
recall_list = []
recall_list_random = []
precision_list = []
precision_list_random = []
f1_scores_r_list = []
f1_scores_r_list_random = []
f1_scores_p_list = []
f1_scores_p_list_random = []
storate_reduction_list = []
beta_r = 2
beta_p = 0.5
# grab the "label" column of each CSV_PATH in the for loop and add it to the end of CSV_PATH_PNGS as "label"+str(threshold)
OUT_PATH_RANDOM = f"/scratch/ahmad9/caserm/detect_mse_and_density/model1600/512.256.4/results_thresh/evaluate/good_random/sampled_lines_eval.csv"  # <- fixed f-string
results_random = pd.read_csv(OUT_PATH_RANDOM)
OUT_PATH_ANOMALOUS = f"/scratch/ahmad9/caserm/detect_mse_and_density/model1600/512.256.4/anomalous/results_thresh/evaluate/sampled_lines_eval.csv"  # <- fixed f-string
results_anomalous = pd.read_csv(OUT_PATH_ANOMALOUS)

# Ensure both DataFrames share the same set of columns (union), filling missing with NaN
all_cols = sorted(set(results_random.columns) | set(results_anomalous.columns)) 
results_random = results_random.reindex(columns=all_cols)
for col in ("label", "evaluate", "seismic"):
    results_random[col] = results_random[col].str.strip().str.lower()
pred_map = {"y": "anomaly", "n": "normal"}
results_random["pred"] = results_random["evaluate"].map(pred_map)

results_anomalous = results_anomalous.reindex(columns=all_cols)

results_all = pd.concat([results_random, results_anomalous], ignore_index=True)

# Read and normalize
# df_ = pd.read_csv(OUT_PATH, dtype=str)
for col in ("label", "evaluate", "seismic"):
    results_all[col] = results_all[col].str.strip().str.lower()


# Map evaluation -> predicted label
pred_map = {"y": "anomaly", "n": "normal"}
results_all["pred"] = results_all["evaluate"].map(pred_map)

# Confusion counts
tp = int(((results_all["label"] == "anomaly") & (results_all["pred"] == "anomaly")).sum())
tp_random = int(((results_random["label"] == "anomaly") & (results_random["pred"] == "anomaly")).sum())
tn = int(((results_all["label"] == "normal")  & (results_all["pred"] == "normal")).sum())
tn_random = int(((results_random["label"] == "normal")  & (results_random["pred"] == "normal")).sum())
fp = int(((results_all["label"] == "anomaly") & (results_all["pred"] == "normal")).sum())
fp_random = int(((results_random["label"] == "anomaly") & (results_random["pred"] == "normal")).sum())
fn = int(((results_all["label"] == "normal")  & (results_all["pred"] == "anomaly")).sum())
fn_random = int(((results_random["label"] == "normal")  & (results_random["pred"] == "anomaly")).sum())
events = int(((results_all["label"] == "anomaly") & (results_all["seismic"] == "y")).sum()) 

print(f"TP={tp}, TN={tn}, FP={fp}, FN={fn}, SEISMIC={events}")

precision = tp / (tp + fp)
precision_random = tp_random / (tp_random + fp_random)
recall = tp / (tp + fn)
recall_random = tp_random / (tp_random + fn_random)

if tp==0:
    f1_score = 0
    f1_score_r = 0
    f1_score_p = 0
    seismic_accuracy = 0
    seismic_to_all_anom = 0
elif tp_random==0:
    storate_reduction = 0
    f1_score_random = 0
    f1_score_r_random = 0
    f1_score_p_random = 0
else:
    f1_score = 2 * precision * recall / (precision +  recall)
    f1_score_random = 2 * precision * recall / (precision +  recall)
    f1_score_r = (1+beta_r**2) * precision * recall / (((beta_r**2) * precision) +  recall)   
    f1_score_r_random = (1+beta_r**2) * precision * recall / (((beta_r**2) * precision) +  recall)  
    f1_score_p = (1+beta_p**2) * precision * recall / (((beta_p**2) * precision) +  recall) 
    f1_score_p_random = (1+beta_p**2) * precision * recall / (((beta_p**2) * precision) +  recall) 
    storate_reduction = results_random.shape[0] / tp_random 
    # seismic_accuracy = events / tp
    seismic_accuracy = events / int((results_all["seismic"] == "y").sum()) 
    seismic_to_all_anom = events / int((results_all["label"] == "anomaly").sum()) 

print(f"Precision = {precision}, Recall = {recall}")
print(f"F1 Score = {f1_score}, Seismic Accuracy = {seismic_accuracy}")
f1_scores_list.append(f1_score)            
f1_scores_list_random.append(f1_score_random) 
seismic_acc_list.append(seismic_accuracy) 
recall_list.append(recall)
precision_list.append(precision)
f1_scores_r_list.append(f1_score_r)
f1_scores_r_list_random.append(f1_score_r_random)
f1_scores_p_list.append(f1_score_p)
f1_scores_p_list_random.append(f1_score_p_random)
storate_reduction_list.append(storate_reduction)
seismic_to_all_anom_list.append(seismic_to_all_anom)
# (Optional) nice confusion table
conf = pd.crosstab(results_all["label"], results_all["pred"]).reindex(
    index=["anomaly", "normal"], columns=["anomaly", "normal"]
).fillna(0).astype(int)
conf.index.name = "True"
conf.columns.name = "Pred"
display(conf)


In [ ]:
f1_scores = np.asarray(f1_scores_list, dtype=float)
f1_scores_random = np.asarray(f1_scores_list_random, dtype=float)
seismic_accuracies = np.asarray(seismic_acc_list, dtype=float)
recalls = np.asarray(recall_list, dtype=float)
precisions = np.asarray(precision_list, dtype=float)
f1_scores_r = np.asarray(f1_scores_r_list, dtype=float)
f1_scores_r_random = np.asarray(f1_scores_r_list_random, dtype=float)
f1_scores_p = np.asarray(f1_scores_p_list, dtype=float)
f1_scores_p_random = np.asarray(f1_scores_p_list_random, dtype=float)
storate_reductions = np.asarray(storate_reduction_list, dtype=float)
seismic_to_all_anom = np.asarray(seismic_to_all_anom_list, dtype=float)

In [ ]:

density_anomaly_txt = "/home/ahmad9/casermminefiber/event_detection_July_2025/high_freq_low_gl/das-anomaly/examples/validate_and_plot_density/scripts/outputs/anomalous_values_model1600_512_256_4.txt"
density_normal_txt = "/home/ahmad9/casermminefiber/event_detection_July_2025/high_freq_low_gl/das-anomaly/examples/validate_and_plot_density/scripts/outputs/normal_values_model1600_512_256_4.txt"

density_anomaly = np.array(np.loadtxt(density_anomaly_txt))  
density_normal = np.array(np.loadtxt(density_normal_txt))   
merged = np.concatenate([density_anomaly.ravel(), density_normal.ravel()]) 

density = np.array([8770, 500, -346])
# density = np.array(thresholds)
density_scaled = (density - merged.min()) / (merged.max() - merged.min())

In [ ]:


fig, ax = plt.subplots(figsize=(10, 8))

# exactly 5 filled markers (good contrast with edgecolor)
markers = ['o', 's', '^', 'D', 'v']

# exactly 4 colors (from current Matplotlib theme)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:4]

for i, (x, y, thr) in enumerate(zip(seismic_accuracies, f1_scores, np.round(density_scaled, 3))):
    m = markers[i % 5]          # cycles every point: 0..4, 0..4, ...
    c = colors[(i // 5) % 4]    # changes every 5 points: color 0,1,2,3

    ax.scatter(
        x, y,
        marker=m,
        s=90,
        facecolor=c,
        edgecolor='black',
        linewidth=0.6,
        label=f"{thr}"
    )

ax.set_xlabel('Seismic accuracy (TP&Seismic/Events)', size=12)
ax.set_ylabel('F1 score', size=12)
ax.grid(True, alpha=0.3)

# (optional) many labels can clutter; uncomment for a compact legend:
# handles, labels = ax.get_legend_handles_labels()
# ax.legend(handles[:10], labels[:10], title="Density score threshold (normalized)", ncol=2)

ax.legend(title="Density score threshold (normalized)")
plt.show()


In [ ]:

fig, ax = plt.subplots(figsize=(10, 8))

# exactly 5 filled markers (good contrast with edgecolor)
markers = ['o', 's', '^', 'D', 'v']

# exactly 4 colors (from current Matplotlib theme)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:4]

for i, (x, y, thr) in enumerate(zip(seismic_accuracies, f1_scores_p, np.round(density_scaled, 3))):
    m = markers[i % 5]          # cycles every point: 0..4, 0..4, ...
    c = colors[(i // 5) % 4]    # changes every 5 points: color 0,1,2,3

    ax.scatter(
        x, y,
        marker=m,
        s=90,
        facecolor=c,
        edgecolor='black',
        linewidth=0.6,
        label=f"{thr}"
    )

ax.set_xlabel('Seismic accuracy (TP&Seismic/Events)', size=12)
ax.set_ylabel('F1 score_p', size=12)
ax.grid(True, alpha=0.3)

# Add dashed rectangle for region 0.9 < x < 1, 0.9 < y < 1
rect = patches.Rectangle(
    (0.9, 0.9),   # bottom-left corner
    0.1, 0.1,     # width and height
    linewidth=1.5,
    edgecolor='red',
    facecolor='none',
    linestyle='--'
)
ax.add_patch(rect)

ax.legend(title="Density score threshold (normalized)")
plt.show()


In [ ]:


fig, ax = plt.subplots(figsize=(10, 8))

# exactly 5 filled markers (good contrast with edgecolor)
markers = ['o', 's', '^', 'D', 'v']

# exactly 4 colors (from current Matplotlib theme)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:4]

for i, (x, y, thr) in enumerate(zip(seismic_accuracies, storate_reductions, np.round(density_scaled, 3))):
    m = markers[i % 5]          # cycles every point: 0..4, 0..4, ...
    c = colors[(i // 5) % 4]    # changes every 5 points: color 0,1,2,3

    ax.scatter(
        x, y,
        marker=m,
        s=90,
        facecolor=c,
        edgecolor='black',
        linewidth=0.6,
        label=f"{thr}"
    )

ax.set_xlabel('Seismic accuracy (TP&Seismic/Events)', size=12)
ax.set_ylabel('Storage reduction factor', size=12)
ax.grid(True, alpha=0.3)

# (optional) many labels can clutter; uncomment for a compact legend:
# handles, labels = ax.get_legend_handles_labels()
# ax.legend(handles[:10], labels[:10], title="Density score threshold (normalized)", ncol=2)

ax.legend(title="Density score threshold (normalized)")
plt.show()


In [ ]:


fig, ax = plt.subplots(figsize=(10, 8))

# exactly 5 filled markers (good contrast with edgecolor)
markers = ['o', 's', '^', 'D', 'v']

# exactly 4 colors (from current Matplotlib theme)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:4]

for i, (x, y, thr) in enumerate(zip(seismic_accuracies, seismic_to_tps, np.round(density_scaled, 3))):
    m = markers[i % 5]          # cycles every point: 0..4, 0..4, ...
    c = colors[(i // 5) % 4]    # changes every 5 points: color 0,1,2,3

    ax.scatter(
        x, y,
        marker=m,
        s=90,
        facecolor=c,
        edgecolor='black',
        linewidth=0.6,
        label=f"{thr}"
    )

ax.set_xlabel('Seismic accuracy (Detected_seismic_events/Events) - Less missing seismic events', size=12)
ax.set_ylabel('Detected_seismic_events/TP - Less false trigger for non-seismic events', size=12)
ax.grid(True, alpha=0.3)

# (optional) many labels can clutter; uncomment for a compact legend:
# handles, labels = ax.get_legend_handles_labels()
# ax.legend(handles[:10], labels[:10], title="Density score threshold (normalized)", ncol=2)

ax.legend(title="Density score threshold (normalized)")
plt.show()


In [ ]:


fig, ax = plt.subplots(figsize=(10, 8))

# exactly 5 filled markers (good contrast with edgecolor)
markers = ['o', 's', '^', 'D', 'v']

# exactly 4 colors (from current Matplotlib theme)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:4]

for i, (x, y, thr) in enumerate(zip(seismic_accuracies, seismic_to_all_anom, np.round(density_scaled, 3))):
    m = markers[i % 5]          # cycles every point: 0..4, 0..4, ...
    c = colors[(i // 5) % 4]    # changes every 5 points: color 0,1,2,3

    ax.scatter(
        x, y,
        marker=m,
        s=90,
        facecolor=c,
        edgecolor='black',
        linewidth=0.6,
        label=f"{thr}"
    )

ax.set_xlabel('Seismic accuracy (Detected_seismic_events/Events) - Less missing seismic events', size=12)
ax.set_ylabel('Detected_seismic_events/TP - Less false trigger for non-seismic events', size=12)
ax.grid(True, alpha=0.3)

# (optional) many labels can clutter; uncomment for a compact legend:
# handles, labels = ax.get_legend_handles_labels()
# ax.legend(handles[:10], labels[:10], title="Density score threshold (normalized)", ncol=2)

ax.legend(title="Density score threshold (normalized)")
plt.show()


In [ ]:

fig, ax = plt.subplots(figsize=(10, 8))

# Ensure arrays are numpy arrays
x = np.asarray(seismic_accuracies)   # (kept your variable name)
y = np.asarray(seismic_to_all_anom)
z = np.asarray(density_scaled)       # normalized density score (0–1), or any range

vmin, vmax = 0.45, 0.5        # pick the band where most points live
norm = plt.Normalize(vmin=vmin, vmax=vmax, clip=True)

sc = ax.scatter(x, y, c=z, cmap='plasma', norm=norm,
                s=90, marker='o', edgecolor='black', linewidth=0.6)
cbar = plt.colorbar(sc, ax=ax, extend='both')  # arrows show clipping
cbar.set_label('Density score (normalized)', size=14)


ax.set_xlabel('Seismic event detection coverage', size=14)
ax.set_ylabel('Seismic event detection precision', size=14)
ax.grid(True, alpha=0.3)

# (optional) many labels can clutter; uncomment for a compact legend:
# handles, labels = ax.get_legend_handles_labels()
# ax.legend(handles[:10], labels[:10], title="Density score threshold (normalized)", ncol=2)

# ax.legend(title="Density score threshold (normalized)")
plt.show()


In [ ]:


fig, ax = plt.subplots(figsize=(10, 8))

# exactly 5 filled markers (good contrast with edgecolor)
markers = ['o', 's', '^', 'D', 'v']

# exactly 4 colors (from current Matplotlib theme)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:4]

for i, (x, y, thr) in enumerate(zip(storate_reductions, f1_scores_r, np.round(density_scaled, 3))):
    m = markers[i % 5]          # cycles every point: 0..4, 0..4, ...
    c = colors[(i // 5) % 4]    # changes every 5 points: color 0,1,2,3

    ax.scatter(
        x, y,
        marker=m,
        s=90,
        facecolor=c,
        edgecolor='black',
        linewidth=0.6,
        label=f"{thr}"
    )

ax.set_xlabel('Storage reduction factor', size=12)
ax.set_ylabel('F1 score_r', size=12)
ax.grid(True, alpha=0.3)

# (optional) many labels can clutter; uncomment for a compact legend:
# handles, labels = ax.get_legend_handles_labels()
# ax.legend(handles[:10], labels[:10], title="Density score threshold (normalized)", ncol=2)

ax.legend(title="Density score threshold (normalized)")
plt.show()



In [ ]:


fig, ax = plt.subplots(figsize=(10, 8))

# exactly 5 filled markers (good contrast with edgecolor)
markers = ['o', 's', '^', 'D', 'v']

# exactly 4 colors (from current Matplotlib theme)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:4]

for i, (x, y, thr) in enumerate(zip(storate_reductions, f1_scores_r_random, np.round(density_scaled, 3))):
    if x == 0:
        continue
    m = markers[i % 5]          # cycles every point: 0..4, 0..4, ...
    c = colors[(i // 5) % 4]    # changes every 5 points: color 0,1,2,3

    ax.scatter(
        x, y,
        marker=m,
        s=90,
        facecolor=c,
        edgecolor='black',
        linewidth=0.6,
        label=f"{thr}"
    )

ax.set_xlabel('Storage reduction factor', size=12)
ax.set_ylabel('F1 score_r', size=12)
ax.grid(True, alpha=0.3)

# (optional) many labels can clutter; uncomment for a compact legend:
# handles, labels = ax.get_legend_handles_labels()
# ax.legend(handles[:10], labels[:10], title="Density score threshold (normalized)", ncol=2)

ax.legend(title="Density score threshold (normalized)")
plt.show()



In [ ]:


fig, ax = plt.subplots(figsize=(10, 8))

# exactly 5 filled markers (good contrast with edgecolor)
markers = ['o', 's', '^', 'D', 'v']

# exactly 4 colors (from current Matplotlib theme)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:4]

for i, (x, y, thr) in enumerate(zip(storate_reductions, f1_scores_p, np.round(density_scaled, 3))):
    m = markers[i % 5]          # cycles every point: 0..4, 0..4, ...
    c = colors[(i // 5) % 4]    # changes every 5 points: color 0,1,2,3

    ax.scatter(
        x, y,
        marker=m,
        s=90,
        facecolor=c,
        edgecolor='black',
        linewidth=0.6,
        label=f"{thr}"
    )

ax.set_xlabel('Storage reduction factor', size=12)
ax.set_ylabel('F1 score_p', size=12)
ax.grid(True, alpha=0.3)

# (optional) many labels can clutter; uncomment for a compact legend:
# handles, labels = ax.get_legend_handles_labels()
# ax.legend(handles[:10], labels[:10], title="Density score threshold (normalized)", ncol=2)

ax.legend(title="Density score threshold (normalized)")
plt.show()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.colors import PowerNorm

fig, ax = plt.subplots(figsize=(10, 8))

# Ensure arrays are numpy arrays
x = np.asarray(storate_reductions)   # (kept your variable name)
y = np.asarray(f1_scores_r)
z = np.asarray(density_scaled)       # normalized density score (0–1), or any range

vmin, vmax = 0.45, 0.5        # pick the band where most points live
norm = plt.Normalize(vmin=vmin, vmax=vmax, clip=True)

sc = ax.scatter(x, y, c=z, cmap='plasma', norm=norm,
                s=90, marker='o', edgecolor='black', linewidth=0.6)
cbar = plt.colorbar(sc, ax=ax, extend='both')  # arrows show clipping
cbar.set_label('Density score (normalized)', size=14)
# Continuous colormap driven by density score
# norm = Normalize(vmin=np.nanmin(z), vmax=np.nanmax(z))
# sc = ax.scatter(
#     x, y,
#     c=z, cmap='viridis', norm=norm,   # change 'viridis' if you prefer
#     s=90, marker='o',
#     edgecolor='black', linewidth=0.6
# )

ax.set_xlabel('Estimated storage reduction factor', size=14)
ax.set_ylabel('F1 score (beta=2)', size=14)
ax.grid(True, alpha=0.3)

# Colorbar instead of legend
# cbar = plt.colorbar(sc, ax=ax)
# cbar.set_label('Density score (normalized)')

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.colors import PowerNorm

fig, ax = plt.subplots(figsize=(10, 8))

# Ensure arrays are numpy arrays
x = np.asarray(storate_reductions)   # (kept your variable name)
y = np.asarray(f1_scores_p)
z = np.asarray(density_scaled)       # normalized density score (0–1), or any range

vmin, vmax = 0.45, 0.5        # pick the band where most points live
norm = plt.Normalize(vmin=vmin, vmax=vmax, clip=True)

sc = ax.scatter(x, y, c=z, cmap='plasma', norm=norm,
                s=90, marker='o', edgecolor='black', linewidth=0.6)
cbar = plt.colorbar(sc, ax=ax, extend='both')  # arrows show clipping
cbar.set_label('Density score (normalized)', size=14)
# Continuous colormap driven by density score
# norm = Normalize(vmin=np.nanmin(z), vmax=np.nanmax(z))
# sc = ax.scatter(
#     x, y,
#     c=z, cmap='viridis', norm=norm,   # change 'viridis' if you prefer
#     s=90, marker='o',
#     edgecolor='black', linewidth=0.6
# )

ax.set_xlabel('Estimated storage reduction factor', size=14)
ax.set_ylabel('F1 score (beta=0.25)', size=14)
ax.grid(True, alpha=0.3)

# Colorbar instead of legend
# cbar = plt.colorbar(sc, ax=ax)
# cbar.set_label('Density score (normalized)')

plt.tight_layout()
plt.show()


In [ ]:


fig, ax = plt.subplots(figsize=(10, 8))

# exactly 5 filled markers (good contrast with edgecolor)
markers = ['o', 's', '^', 'D', 'v']

# exactly 4 colors (from current Matplotlib theme)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color'][:4]

for i, (x, y, thr) in enumerate(zip(recalls, precisions, np.round(density_scaled, 3))):
    m = markers[i % 5]          # cycles every point: 0..4, 0..4, ...
    c = colors[(i // 5) % 4]    # changes every 5 points: color 0,1,2,3

    ax.scatter(
        x, y,
        marker=m,
        s=90,
        facecolor=c,
        edgecolor='black',
        linewidth=0.6,
        label=f"{thr}"
    )

ax.set_xlabel('Recall', size=12)
ax.set_ylabel('Precision', size=12)
ax.grid(True, alpha=0.3)

# (optional) many labels can clutter; uncomment for a compact legend:
# handles, labels = ax.get_legend_handles_labels()
# ax.legend(handles[:10], labels[:10], title="Density score threshold (normalized)", ncol=2)

ax.legend(title="Density score threshold (normalized)")
plt.show()
